<a href="https://colab.research.google.com/github/hussainturii/Advanced-Computer-Vision---Deep-Learning/blob/main/Transfer_Learning_without_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import wandb
from wandb.integration.keras import WandbMetricsLogger

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [2]:
# Sweep configuration
sweep_config = {
    'method': 'grid',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'batch_size': {'values': [8]},
        'learning_rate': {'values': [0.0001]},
        'img_size': {'values': [128]},  # Set a reasonably large size for MobileNetV2
        'epochs': {'values': [10]},
        'experiment': {'values': ['transfer_learning']}
    }
}

In [3]:
# Initialize W&B sweep
sweep_id = wandb.sweep(sweep_config, project="5-flowers-transfer-learning")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 2ygch0n0
Sweep URL: https://wandb.ai/hussaintoori/5-flowers-transfer-learning/sweeps/2ygch0n0


In [4]:
# Train function
def train():
    with wandb.init() as run:
        config = wandb.config

        IMG_HEIGHT = config.img_size
        IMG_WIDTH = config.img_size
        IMG_CHANNELS = 3
        CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

        # Helper functions
        def read_and_decode(filename, resize_dims):
            img_bytes = tf.io.read_file(filename)
            img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
            img = tf.image.convert_image_dtype(img, tf.float32)
            img = tf.image.resize(img, resize_dims)
            return img

        def parse_csvline(csv_line):
            record_default = ["", ""]
            filename, label_string = tf.io.decode_csv(csv_line, record_default)
            img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
            label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]
            return img, label

        # Prepare datasets
        train_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        eval_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        # Build model
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
            include_top=False,    # Remove ImageNet head
            weights="imagenet"    # Use pretrained weights
        )
        base_model.trainable = False  # Freeze base

        model = keras.Sequential([
            base_model,
            keras.layers.GlobalAveragePooling2D(),
            keras.layers.BatchNormalization(),       # Add BatchNorm
            keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
        ])

        # Compile model
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=config.learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=["accuracy"]
        )

        # Visualize model
        model.summary()

        keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file="model_visualization.png")

        # Train
        callbacks = [WandbMetricsLogger(log_freq=5)]

        model.fit(
            train_dataset,
            validation_data=eval_dataset,
            epochs=config.epochs,
            callbacks=callbacks
        )

from above: we uses an already trained model from MobileNetV2 architecture. the model is **imageNet**
**MobileNetV2** is a computer vision architecture by Google that efficiently classifies images on mobile and embedded devices by using inverted residual blocks with linear bottlenecks, reducing computational cost while maintaining accuracy

*base_model.trainable = False  # Freeze base*
In this code line we opt to use only the base model of the MobileNetV2, meaning Only the new layers you add on top (like Dense layers for classification) will be trained.

we also apply BatchNorm and GlobalAveragePool (Input: A 3D feature map (Height × Width × Channels).
Output: A 1D vector (just the average per channel).)

In [5]:
# Launch W&B agent
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: qgwq6hab with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: transfer_learning
wandb: 	img_size: 128
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: hussainturi (hussaintoori) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         6,405 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,269,509 (8.66 MB)

...

Epoch 1/10
    413/Unknown 323s 765ms/step - accuracy: 0.3259 - loss: 1.8201

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 361s 856ms/step - accuracy: 0.3263 - loss: 1.8192 - val_accuracy: 0.6784 - val_loss: 0.9397
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 336s 746ms/step - accuracy: 0.6738 - loss: 0.8614 - val_accuracy: 0.7676 - val_loss: 0.6902
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 275s 633ms/step - accuracy: 0.7552 - loss: 0.6748 - val_accuracy: 0.8027 - val_loss: 0.5957
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 234s 566ms/step - accuracy: 0.7847 - loss: 0.5852 - val_accuracy: 0.8135 - val_loss: 0.5446
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 216s 522ms/step - accuracy: 0.8121 - loss: 0.5271 - val_accuracy: 0.8243 - val_loss: 0.5118
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 209s 506ms/step - accuracy: 0.8344 - loss: 0.4839 - val_accuracy: 0.8297 - val_loss: 0.4886
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 200s 485ms/step - accuracy: 0.8458 - loss: 0.4492 - val_accuracy: 0.8378 - val_loss: 0.4714
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 201s 487ms/step - accuracy: 0.8538 - loss: 0.42

batch/accuracy,▁▁▁▂▃▆▆▆▇▇▇▇▇▇▇▇▇█▇▇█████▇████████▇█████
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,███▆▆▅▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▅▆▇▇▇▇███
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▂▂▂▁▁▁▁
epoch/val_accuracy,▁▅▆▇▇▇████
epoch/val_loss,█▅▃▂▂▂▁▁▁▁
batch/accuracy,0.87409


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
